In [4]:
import os
import pyspark
import numpy as np
import pandas as pd
import itertools as it
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
import time
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

In [ ]:
def get_first_day(dt, d_years=0, d_months=0):
    # d_years, d_months are "deltas" to apply to dt
    y, m = dt.year + d_years, dt.month + d_months
    a, m = divmod(m-1, 12)
    return date(y+a, m+1, 1)

In [ ]:
def outlier_removal(data, signal):
    group_level = ["servicing_npi", "claim_number"]
    signal_list = [c for c in signal if "ind" not in c]
    col_to_work = signal_list + group_level
    
    data_to_work = data.select(col_to_work)
    for c in signal_list:
        c_ucl = c + "_UCL"
        c_lcl = c + "_LCL"
        c_flg = c + "_flag"
        op = control_limit(data_to_work, c, k)
        data_to_work = data_to_work.join(op, on = "servicing_npi", how = "inner")
        data_to_work = data_to_work.withColumn(c_flg, F.when(((data_to_work[c] <= data_to_work[c_ucl]) & (data_to_work[c] >= data_to_work[c_lcl])), F.lit(1)).otherwise(F.lit(0)))
        data_to_work = data_to_work.drop(c_ucl, c_lcl)
        
    data_to_work_outlier_removed = data_to_work.filter((F.col('billed_amt_sum_flag') == F.lit(1)) &
                                                    (F.col('clm_ln_cnt_flag') == F.lit(1)) &
                                                    (F.col('clm_unit_billed_flag') == F.lit(1)) &
                                                    (F.col('proc_code_unusual_flag') == F.lit(1)) &
                                                    (F.col('pos_unsual_flag') == F.lit(1)))
    data_to_work_outlier_removed = data_to_work_outlier_removed.select(group_level)
    modelling_data = data.join(data_to_work_outlier_removed, on = group_level, how = "inner")
    
    return modelling_data

In [106]:
def control_limit(data, c, k):
    print("Working on {}".format(c))
    col_list = ['servicing_npi', c]
    temp_df = data.select(col_list)
    temp_df.createOrReplaceTempView("t1")
    c_q1 = c + "_Q1"
    c_q3 = c + "_Q3"
    c_iqr = c + "_IQR"
    c_ucl = c + "_UCL"
    c_lcl = c + "_LCL"
    query =  "select servicing_npi, percentile({}, 0.25) as {}, percentile({}, 0.75) as {} from t1 group by servicing_npi".format(c, c_q1, c, c_q3)
    query = " ".join(query.split())
    op = get_data(query)
    op = op.withColumn(c_iqr, op[c_q3] - op[c_q1])
    op = op.withColumn(c_lcl, op[c_q1] - (op[c_iqr]*k))
    op = op.withColumn(c_ucl, op[c_q3] + (op[c_iqr]*k))
    op = op.select(['servicing_npi', c_ucl, c_lcl])
    return op

In [6]:
def get_data(query):
    """
    Returns the pySpark DataFrame
    
    Parameters:
        query (str): The input HQL query
        
    Returns:
        DataFrame (pySpark)
    """
    try:
        return sessn.sql(query)
    except Exception as e:
        print(e)

In [ ]:
def lookback_data(data, grouping_entity, entity):
    data = data.filter(~((data[entity] == "")|(data[entity].isNull())))
    df_agg = data.groupby(grouping_entity, entity).agg(F.count(entity).alias("freq"))
    df_agg_npi = df.groupby(grouping_entity).agg(F.count(entity).alias("freq_sum"))
    df_agg =  df_agg.join(df_agg_npi, on = grouping_entity, how = "inner")
    df_agg = df_agg.withColumn("freq_dist", F.round(F.col("freq")/F.col("freq_sum"), 3)).drop("freq", "freq_sum")
    df_agg.write.mode("overwrite").saveAsTable("pi_stg.psm_lb_"+entity)

In [ ]:
@udf("int")
def place_of_srcv_validator(pls):
    """
    Check for valid place of service (01 to 99)
    
    Parameters:
        pls (string)
    
    Returns:
        1 if the place of service is valid else 0.
    """
    try:
        if (pls in pls_lookup_list):
            return 1
        else:
            return 0
    except:
        return 0  

In [107]:
# Configuration
# Input
feature_engineered_table = "pi_stg.ps_history"
modelling_table = "pi_stg_tmp.ps_modelling_data"
k = 1.5
lb_dur = 3

signal_list =  ['billed_amt_sum',
                 'clm_ln_cnt',
                 'clm_unit_billed',
                 'proc_code_unusual',
                 'pos_unsual',
                 'em_proc_ind',
                 'mod91_ind',
                 'modTC_ind',
                 'mod59_ind',
                 'high_cst_proc_ind',
                 'emgncy_ind']

clm_th =10

mean_table = "psm_mean_data"
icov_table = "psm_icov_data"

In [188]:
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '12g'), 
                                   ('spark.executor.instances', '24'),
                                   ('spark.executor.cores', '4'), 
                                   ('spark.driver.cores','4'), 
                                   ('spark.driver.memory','25g'),
                                   ('spark.yarn.queue', 'araadh_q1.arapi_sq1'),
                                   ('spark.yarn.executor.memoryOverhead', '4g')])
sessn = SparkSession.builder.appName("PS").config(conf=conf).enableHiveSupport().getOrCreate().newSession()
sessn

***Dropping older partitions***

In [ ]:
drop_start_date = get_first_day(date.today() - relativedelta(months = 13)).strftime('%Y%m%d')
drop_end_date = get_first_day(date.today() - relativedelta(months = 12)).strftime('%Y%m%d')
drop_dates = []
for dt in range(int(drop_start_date),int(drop_end_date)-1):
    drop_dates.append(str(dt))
for dp in drop_dates:
    print("Dropping Partition for : ",dp)
    sessn.sql("""ALTER TABLE {} DROP IF EXISTS PARTITION(batch_run_dt = {})""".format(feature_engineered_table,dp))

#### Outlier Removal

In [85]:
base_data = sessn.table(feature_engineered_table)
modelling_data = outlier_removal(base_data, signal_list)
modelling_data.write.mode("overwrite").saveAsTable(modelling_table)

In [ ]:
#### Lookback Table Creation

In [1]:
start_date = get_first_day(date.today() - relativedelta(months = lb_dur)).strftime('%Y%m%d')
end_date = get_first_day(date.today()).strftime('%Y%m%d')
query = """SELECT DISTINCT srk_clm_id,  
    CASE WHEN trim(cl_rend_prov_npi) = '' OR trim(cl_rend_prov_npi) IS NULL THEN trim(clm_bill_prov_npi) ELSE trim(cl_rend_prov_npi) END AS servicing_npi, 
    batch_run_dt, trim(cl_proc_cd) as cl_proc_cd, trim(cl_place_of_srcv) as cl_place_of_srcv FROM pi_prod.837p_clm_analytic_data
    where batch_run_dt >= '{}' AND batch_run_dt <'{}'
    """.format(start_date, end_date)
query = " ".join(query.split())

df = sessn.sql(query)
global pls_lookup_list
pls_lookup_list = ["0" + str(i) for i in range(1, 10)] + [str(i) for i in range(1,100)]
df1 = df.withColumn("valid_pls", place_of_srcv_validator("cl_place_of_srcv"))
df1 = df1.filter(df1["valid_pls"] == 1)
df1 = df1.withColumn("cl_place_of_srcv", F.lpad(df1["cl_place_of_srcv"], 2, '0'))

lookback_data(df, "servicing_npi", "cl_proc_cd")
lookback_data(df1, "servicing_npi", "cl_place_of_srcv")

NameError: name 'get_first_day' is not defined

In [ ]:
### Distribution Creation

In [ ]:
modelling_data = sessn.table(modelling_table)

# Generating list Good NPIs (Having more than 10 Claims in Modelling data)
npi_clm_cnt = modelling_data.groupby("servicing_npi").count()
good_npi_clm = npi_clm_cnt.filter(F.col("count")>=clm_th)
good_npi = good_npi_clm.select("servicing_npi")
scorable_npi_count = good_npi.count()
# data_to_work_modelling -> Consist modelling data for scorable NPIs
data_to_work_modelling = modelling_data.join(good_npi, on ="servicing_npi", how ="inner")

scnt = len(signal_list)
signal_comb = list(it.product(signal_list, repeat =2 ))

covarince_op = data_to_work_modelling.groupby("servicing_npi").agg(*[F.covar_samp(f[0],f[1]) for f in signal_comb])
mean_op = data_to_work_modelling.groupBy("servicing_npi").agg(*[F.mean(f) for f in signal_list])

covarince_op_pd = covarince_op.toPandas()

covar_dump = covarince_op_pd.values[:,1:]
covar_dump = covar_dump.reshape(scorable_npi_count, scnt, scnt)
col_list = list(covarince_op_pd.columns[1:])

In [ ]:
# Computing Pseudo Inverse of covariance matrix
final_list = []    
for npi in range(scorable_npi_count):
    temp_icov = np.linalg.pinv((covar_dump[npi]).astype(float))
    temp_icov = temp_icov.flatten().tolist() 
    final_list.append(temp_icov)

In [ ]:
icovariance_op_pd = pd.DataFrame(final_list, columns = col_list)
icovariance_op_pd['servicing_npi'] = covarince_op_pd['servicing_npi']
icovariance_op_pd = icovariance_op_pd[list(covarince_op_pd)]

In [ ]:
icovariance_op = sessn.createDataFrame(icovariance_op_pd)

for s in signal_list:
    s_col_ = "avg({})".format(s)
    s_col_op = "avg_{}".format(s)
    mean_op = mean_op.withColumnRenamed(s_col_, s_col_op)
for s_c in signal_comb:
    s_col_ = "covar_samp({}, {})".format(s_c[0], s_c[1])
    s_col_op = "covar_samp_{}_{}".format(s_c[0], s_c[1])
    icovariance_op = icovariance_op.withColumnRenamed(s_col_, s_col_op)

In [ ]:
mean_op.write.mode("overwrite").format("orc").saveAsTable(mean_table)
icovariance_op.write.mode("overwrite").format("orc").saveAsTable(icov_table)

In [207]:
sessn.stop()